In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import hvplot.pandas
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report
from finta import TA

### Initial Data Import and Cleaning

In [2]:
# Read the csv file 
amzn_df = pd.read_csv(Path("Resources/amzn.csv"))

# Convert 'time' column from timestamp (seconds since epoch) to acutal time
amzn_df ['time'] = pd.to_datetime(
    amzn_df['time'],
    unit = 's',
    infer_datetime_format=True,
    utc=True
)    
# Convert timezone from UTC to Eastern Time
amzn_df['time'] = amzn_df['time'].dt.tz_convert('US/Eastern')

# Set 'time' column as the index 
amzn_df.set_index('time', inplace=True)

# Review df
amzn_df.head()          

,open,high,low,close,VWAP,Upper Band #1,Lower Band #1,Upper Band #2,Lower Band #2,Upper Band #3,...,Volume,Volume MA,EMA,Smoothing Line,Developing Poc,Developing VA High,Developing VA Low,Developing Poc.1,Developing VA High.1,Developing VA Low.1
time,,,,,,,,,,,,,,,,,,,,,
2012-03-14 09:30:00-04:00,9.1825,9.215995,9.057000,9.112999,9.128665,9.128665,9.128665,9.128665,9.128665,9.128665,...,73994680,107206355.0,9.597910,9.606610,NaN,NaN,NaN,NaN,NaN,NaN
2012-03-15 09:30:00-04:00,9.1010,9.221500,9.015000,9.221500,9.152667,9.152667,9.152667,9.152667,9.152667,9.152667,...,83212340,103564913.0,9.594164,9.602464,NaN,NaN,NaN,NaN,NaN,NaN
2012-03-16 09:30:00-04:00,9.1640,9.284000,9.117501,9.252500,9.218000,9.218000,9.218000,9.218000,9.218000,9.218000,...,98696480,95934268.0,9.590765,9.598431,NaN,NaN,NaN,NaN,NaN,NaN
2012-03-19 09:30:00-04:00,9.1725,9.334000,9.150000,9.276000,9.253333,9.253333,9.253333,9.253333,9.253333,9.253333,...,78080380,92419724.0,9.587633,9.594651,NaN,NaN,NaN,NaN,NaN,NaN
2012-03-20 09:30:00-04:00,9.2440,9.720500,9.144000,9.616500,9.493667,9.493667,9.493667,9.493667,9.493667,9.493667,...,183346180,94828072.0,9.587920,9.591678,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Filter column and only keep ones needed
amzn_df = amzn_df[['open','high','low','close','VWAP','Volume','Volume MA','EMA']]

#Rename EMA to 200EMA
amzn_df = amzn_df.rename(columns={'EMA':'200EMA'})

#Dropping NAN Values 
amzn_df = amzn_df.dropna()

# Review the DataFrame
amzn_df.head()

,open,high,low,close,VWAP,Volume,Volume MA,200EMA
time,,,,,,,,
2012-03-14 09:30:00-04:00,9.1825,9.215995,9.057000,9.112999,9.128665,73994680,107206355.0,9.597910
2012-03-15 09:30:00-04:00,9.1010,9.221500,9.015000,9.221500,9.152667,83212340,103564913.0,9.594164
2012-03-16 09:30:00-04:00,9.1640,9.284000,9.117501,9.252500,9.218000,98696480,95934268.0,9.590765
2012-03-19 09:30:00-04:00,9.1725,9.334000,9.150000,9.276000,9.253333,78080380,92419724.0,9.587633
2012-03-20 09:30:00-04:00,9.2440,9.720500,9.144000,9.616500,9.493667,183346180,94828072.0,9.587920


In [4]:
# Read the csv file 
meta_df = pd.read_csv(Path("Resources/meta.csv"))

# Convert 'time' column from timestamp (seconds since epoch) to acutal time
meta_df ['time'] = pd.to_datetime(
    meta_df['time'],
    unit = 's',
    infer_datetime_format=True,
    utc=True
)    
# Convert timezone from UTC to Eastern Time
meta_df['time'] = meta_df['time'].dt.tz_convert('US/Eastern')

# Set 'time' column as the index 
meta_df.set_index('time', inplace=True)

# Review df
meta_df.head()

,open,high,low,close,VWAP,Upper Band #1,Lower Band #1,Upper Band #2,Lower Band #2,Upper Band #3,...,Volume,Volume MA,EMA,Smoothing Line,Developing Poc,Developing VA High,Developing VA Low,Developing Poc.1,Developing VA High.1,Developing VA Low.1
time,,,,,,,,,,,,,,,,,,,,,
2012-05-18 09:30:00-04:00,42.05,45.00000,38.00,38.2318,40.410600,40.410600,40.410600,40.410600,40.410600,40.410600,...,580587776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-05-21 09:30:00-04:00,36.53,36.66000,33.00,34.0300,34.563333,34.563333,34.563333,34.563333,34.563333,34.563333,...,168309808,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-05-22 09:30:00-04:00,32.61,33.59000,30.94,31.0000,31.843333,31.843333,31.843333,31.843333,31.843333,31.843333,...,102053808,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-05-23 09:30:00-04:00,31.37,32.50000,31.36,32.0000,31.953333,31.953333,31.953333,31.953333,31.953333,31.953333,...,73721120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-05-24 09:30:00-04:00,32.95,33.21001,31.77,33.0300,32.670003,32.670003,32.670003,32.670003,32.670003,32.670003,...,50275872,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Filter column and only keep ones needed
meta_df = meta_df[['open','high','low','close','VWAP','Volume','Volume MA','EMA']]

#Rename EMA to 200EMA
meta_df = meta_df.rename(columns={'EMA':'200EMA'})

#Dropping NAN Values 
meta_df = meta_df.dropna()

# Review the DataFrame
meta_df.head()

,open,high,low,close,VWAP,Volume,Volume MA,200EMA
time,,,,,,,,
2013-03-07 09:30:00-05:00,27.57001,28.67500,27.47000,28.57800,28.241000,74540128,46413816.0,25.669904
2013-03-08 09:30:00-05:00,28.42500,28.47000,27.73000,27.96001,28.053337,44198832,46896756.8,25.692691
2013-03-11 09:30:00-04:00,28.00999,28.64000,27.83000,28.14000,28.203333,35642064,46793423.2,25.717042
2013-03-12 09:30:00-04:00,28.10001,28.32001,27.60001,27.83000,27.916673,27569584,46303815.2,25.738067
2013-03-13 09:30:00-04:00,27.62000,27.64999,26.92000,27.08000,27.216663,39619440,43610005.6,25.751419


In [6]:
# Read the csv file 
tsla_df = pd.read_csv(Path("./Resources/tsla.csv"))

# Convert 'time' column from timestamp (seconds since epoch) to acutal time
tsla_df ['time'] = pd.to_datetime(
    tsla_df['time'],
    unit = 's',
    infer_datetime_format=True,
    utc=True
)    
# Convert timezone from UTC to Eastern Time
tsla_df['time'] = tsla_df['time'].dt.tz_convert('US/Eastern')

# Set 'time' column as the index 
tsla_df.set_index('time', inplace=True)
                      
# Review df
tsla_df.head()   

,open,high,low,close,VWAP,Upper Band #1,Lower Band #1,Upper Band #2,Lower Band #2,Upper Band #3,...,Volume,Volume MA,EMA,Smoothing Line,Developing Poc,Developing VA High,Developing VA Low,Developing Poc.1,Developing VA High.1,Developing VA Low.1
time,,,,,,,,,,,,,,,,,,,,,
2012-03-12 09:30:00-04:00,2.312664,2.419330,2.306665,2.400664,2.375553,2.375553,2.375553,2.375553,2.375553,2.375553,...,29467829,16848705.00,1.929784,1.921918,NaN,NaN,NaN,NaN,NaN,NaN
2012-03-13 09:30:00-04:00,2.433997,2.439331,2.366664,2.405998,2.403998,2.403998,2.403998,2.403998,2.403998,2.403998,...,15024660,16731543.65,1.934522,1.925750,NaN,NaN,NaN,NaN,NaN,NaN
2012-03-14 09:30:00-04:00,2.399998,2.399998,2.319998,2.352664,2.357553,2.357553,2.357553,2.357553,2.357553,2.357553,...,12771972,16011617.15,1.938683,1.929832,NaN,NaN,NaN,NaN,NaN,NaN
2012-03-15 09:30:00-04:00,2.351998,2.365331,2.318664,2.333331,2.339109,2.339109,2.339109,2.339109,2.339109,2.339109,...,8573948,14366955.50,1.942610,1.934130,NaN,NaN,NaN,NaN,NaN,NaN
2012-03-16 09:30:00-04:00,2.326664,2.392664,2.321998,2.354665,2.356442,2.356442,2.356442,2.356442,2.356442,2.356442,...,10938550,13249154.35,1.946710,1.938462,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Filter column and only keep ones needed
tsla_df = tsla_df[['open','high','low','close','VWAP','Volume','Volume MA','EMA']]

#Rename EMA to 200EMA
tsla_df = tsla_df.rename(columns={'EMA':'200EMA'})

#Dropping NAN Values 
tsla_df = tsla_df.dropna()

# Review the DataFrame
tsla_df.head()

,open,high,low,close,VWAP,Volume,Volume MA,200EMA
time,,,,,,,,
2012-03-12 09:30:00-04:00,2.312664,2.419330,2.306665,2.400664,2.375553,29467829,16848705.00,1.929784
2012-03-13 09:30:00-04:00,2.433997,2.439331,2.366664,2.405998,2.403998,15024660,16731543.65,1.934522
2012-03-14 09:30:00-04:00,2.399998,2.399998,2.319998,2.352664,2.357553,12771972,16011617.15,1.938683
2012-03-15 09:30:00-04:00,2.351998,2.365331,2.318664,2.333331,2.339109,8573948,14366955.50,1.942610
2012-03-16 09:30:00-04:00,2.326664,2.392664,2.321998,2.354665,2.356442,10938550,13249154.35,1.946710


### Adding Additional Features

In [8]:
# Read the csv file 
vix_df = pd.read_csv(Path("Resources/vix.csv"))

# Convert 'time' column from timestamp (seconds since epoch) to acutal time
vix_df ['time'] = pd.to_datetime(
    vix_df['time'],
    unit = 's',
    infer_datetime_format=True,
    utc=True
)    
# Convert timezone from UTC to Eastern Time
vix_df['time'] = vix_df['time'].dt.tz_convert('US/Eastern')

# Set 'time' column as the index 
vix_df.set_index('time', inplace=True)

# Calculate VIX EMA 
vix_df['VIX_40_EMA'] = TA.EMA(vix_df,40)

#Remove unnecessary columns
vix_df = vix_df[['close','VIX_40_EMA']]

#Rename close column
vix_df = vix_df.rename(columns={'close':'VIX_close'})

# Review df
vix_df.head()    

,VIX_close,VIX_40_EMA
time,,
2009-10-13 09:30:00-04:00,22.98999,22.989990
2009-10-14 09:30:00-04:00,22.85999,22.923365
2009-10-15 09:30:00-04:00,21.71999,22.502021
2009-10-16 09:30:00-04:00,21.42999,22.213586
2009-10-19 09:30:00-04:00,21.48999,22.054043


In [9]:
# Add VIX data to stock dfs
AMZN = pd.concat([amzn_df,vix_df],axis=1)
META = pd.concat([meta_df,vix_df],axis=1)
TSLA = pd.concat([tsla_df,vix_df],axis=1)


In [10]:
AMZN

,open,high,low,close,VWAP,Volume,Volume MA,200EMA,VIX_close,VIX_40_EMA
time,,,,,,,,,,
2009-10-13 09:30:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.98999,22.989990
2009-10-14 09:30:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.85999,22.923365
2009-10-15 09:30:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.71999,22.502021
2009-10-16 09:30:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.42999,22.213586
2009-10-19 09:30:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.48999,22.054043
...,...,...,...,...,...,...,...,...,...,...
2023-07-18 09:30:00-04:00,132.71,133.86,131.355,132.83,132.681667,54969133.0,52859640.65,113.656816,13.30000,14.809797
2023-07-19 09:30:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.76000,14.758588
2023-07-20 09:30:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.99000,14.721096


### Trading Algo (Signal Generation)

In [11]:
#set some options for display and troubleshooting
pd.set_option("display.max_rows", 2000)
pd.set_option("display.max_columns", 2000)
pd.set_option("display.width", 1000)

In [12]:
# Define periods for fast and slow EMAs (triggers)
fast_ema = 9
slow_ema = 40

# Initialize list of ticker dfs for use in for loop
ticker_df = [AMZN,META,TSLA]

# For each ticker dataframe in the list:
for df in ticker_df:
    
    # Calculate % returns for later use
    df['pct_returns'] = df['close'].pct_change()
    
    # Calculate Fast EMA 
    df['Fast_EMA'] = TA.EMA(df,fast_ema)
    
    # Calculate Slow EMA
    df['Slow_EMA'] = TA.EMA(df,slow_ema)

    # Initialize empty Signal column with 0.0 default value
    df['Signal'] = 0.0

    # Generate Signal value ("long if fast_ema > slow_ema", otherwise flip short)
    df['Signal'] = np.where(
        df['Slow_EMA'] < df['Fast_EMA'], 1.0, -1.0)
    
#This loop should not generate Entry/Exit - this will not be needed until after ML predictions    
    # Create Entry/Exit column and fill with the differences (trades) from Signal column
    #df['Entry/Exit'] = df['Signal'].diff()
    
    #Drop any NA values
    df.dropna()


In [13]:
# Filter each dataframe to 10yr period
amzn_df_filt = AMZN.loc['05-2012':'05-2022']
meta_df_filt = META.loc['05-2012':'05-2022']
tsla_df_filt = TSLA.loc['05-2012':'05-2022']

In [15]:
tsla_df_filt.tail(3)

,open,high,low,close,VWAP,Volume,Volume MA,200EMA,VIX_close,VIX_40_EMA,pct_returns,Fast_EMA,Slow_EMA,Signal
time,,,,,,,,,,,,,,
2022-05-27 09:30:00-04:00,241.083092,253.266413,240.176793,253.209747,248.884318,89295071.0,92315446.55,289.978212,25.72,27.844726,0.073333,237.103143,275.254309,-1.0
2022-05-30 09:30:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.54,27.781081,0.000000,237.103143,275.254309,-1.0
2022-05-31 09:30:00-04:00,257.946409,259.599740,244.743089,252.753081,252.365303,101914472.0,93622097.85,289.607813,26.19,27.703467,-0.001804,240.829319,274.103279,-1.0


In [21]:
amzn_st = pd.read_csv(Path("Search_Trends/Amazon.csv"))
amzn_st = amzn_st.drop(amzn_st.index[0])
amzn_st = amzn_st.rename(columns={'Category: All categories':'AMZN_trends'})

display(amzn_st.head())

meta_st = pd.read_csv(Path("Search_Trends/Meta.csv"))
meta_st = meta_st.drop(meta_st.index[0])
meta_st = meta_st.rename(columns={'Category: All categories':'META_trends'})

display(meta_st.head())


tsla_st = pd.read_csv(Path("Search_Trends/Tesla.csv"))
tsla_st = tsla_st.drop(tsla_st.index[0])
tsla_st = tsla_st.rename(columns={'Category: All categories':'TSLA_trends'})

display(tsla_st.head())

,AMZN_trends
2012-05,40
2012-06,40
2012-07,41
2012-08,42
2012-09,44


,META_trends
2012-05,87
2012-06,89
2012-07,84
2012-08,82
2012-09,79


,TSLA_trends
2012-05,8
2012-06,8
2012-07,8
2012-08,8
2012-09,8
